### Imports

In [ ]:
import syft as sy
import torch
from tools import models
import numpy as np
import pandas as pd
import time

sy.load('opacus')
np.random.seed(42) # The meaning of life!

### Join duet session (DS side)

In [ ]:
duet = sy.join_duet(loopback=True)

### Getting pointers to data in duet store

In [ ]:
# Getting the pionters to the data
time.sleep(31) # Sleep timer so you can just press restart on both notebooks without caring about loading time (might need to be adjusted)

train_data_ptr = duet.store[0]
train_labels_ptr = duet.store[1]

test_data_ptr = duet.store[2]
test_labels_ptr = duet.store[3]

### Parameters and constants

In [ ]:
# Constants for tracking purposes
MODEL = 'Deep2DNet'
DATASET = 'MedNIST'
TRACKING = False # Whether or not this run should be tracked in the results csv file
DP = True # Whether or not Differential Privacy should be applied

# Parameters for training and Differential Privacy
BATCH_SIZE = 100
EPOCHS = 18
LEARNING_RATE = 0.0025 if DP else 0.002

DELTA = 1e-4 # Set to be less then the inverse of the size of the training dataset (from https://opacus.ai/tutorials/building_image_classifier)
NOISE_MULTIPLIER = 2.0 # The amount of noise sampled and added to the average of the gradients in a batch (from https://opacus.ai/tutorials/building_image_classifier)
MAX_GRAD_NORM = 0.1 # The maximum L2 norm of per-sample gradients before they are aggregated by the averaging step (from https://opacus.ai/tutorials/building_image_classifier)

length = len(train_data_ptr)
SAMPLE_SIZE = length - length % BATCH_SIZE # NOTE: Current implementation only trains data in multiples of batch size. So BATCH_SIZE % LENGTH amount of data will not be used for training.
SAMPLE_RATE = BATCH_SIZE / SAMPLE_SIZE

### Preparation for training

In [ ]:
# Getting remote and local instances
local_model = models.Deep2DNet(torch)
remote_model = local_model.send(duet)
remote_torch = duet.torch
remote_opacus = duet.opacus

# Setting device to train on
cuda_available = remote_torch.cuda.is_available().get(request_block=True, reason='Need to check for available GPU!')
if cuda_available:
    device = remote_torch.device('cuda:0')
    remote_model.cuda(device)
else:
    device = remote_torch.device('cpu')
    remote_model.cpu()

# Optimizer and Loss Function
params = remote_model.parameters()
optim = remote_torch.optim.Adam(params=params, lr=LEARNING_RATE)
loss_function = remote_torch.nn.CrossEntropyLoss()

# Setting up Differential Privacy Engine
if DP:
    privacy_engine_ptr = remote_opacus.privacy_engine.PrivacyEngine(
        remote_model.real_module, sample_rate=SAMPLE_RATE,
        noise_multiplier=NOISE_MULTIPLIER, max_grad_norm=MAX_GRAD_NORM
    )
    privacy_engine_ptr.to(device)
    privacy_engine_ptr.attach(optim)
else:
    privacy_engine_ptr = None


### Training and testing the remote model

In [ ]:
from tools.utils import train, test

losses, test_accs, test_losses, epsilons, alphas, epoch_times = train(BATCH_SIZE, EPOCHS, DELTA, 
                                                                      remote_model, remote_torch,
                                                                      optim, loss_function, 
                                                                      train_data_ptr, train_labels_ptr, 
                                                                      test_data_ptr, test_labels_ptr, 
                                                                      [1, 64, 64], device, privacy_engine_ptr)

### Validating the resulting model

In [ ]:
from tools import datasets
# TODO!: Use best model for validation
# Sadly very redundant since remotly tracking best model is not easily possible and therefore just last model is used for validation
# Evalutating the model locally with the validation data
eval_model = remote_model.get(request_block=True, reason="Needed for local evaluation!")
eval_model.cuda(torch.device('cuda:0'))
loss_function = torch.nn.CrossEntropyLoss()

# Higher sample_size here as on DO side will ensure that it is actually data that is not in train or test set!
_, _, val_ds = datasets.Loader.load_MedNIST(sample_size=0.1, test_size=0.1, val_size=0.05)
val_data, val_labels = val_ds.as_tensor()
print(len(val_data))

val_acc, val_loss = test(eval_model, loss_function, torch, val_data, val_labels, torch.device('cuda:0'))

print(f'Validation Accuracy: {val_acc} ---- Validation Loss: {val_loss}')

### Tracking all parameters

In [ ]:
# Tracking all interesting variables and results in .csv file
if TRACKING:
    d = {
        'model': MODEL,
        'dataset': DATASET,
        'batch_size': BATCH_SIZE,
        'epochs': EPOCHS,
        'learning_rate': LEARNING_RATE,
        'train_sample_size': SAMPLE_SIZE,
        'test_sample_size': len(test_data_ptr),
        'val_sample_size': len(val_data),
        'delta': DELTA,
        'noise_multiplier': NOISE_MULTIPLIER,
        'max_grad_norm': MAX_GRAD_NORM,
        'dp_used': DP,
        'epsilons': epsilons,
        'alphas': alphas,
        'train_losses': losses,
        'test_accs': test_accs,
        'test_losses': test_losses,
        'val_acc': val_acc,
        'val_loss': val_loss,
        'epoch_times': epoch_times
    }      
    df = pd.read_csv('./Results/1DO-1DS.csv')
    df = df.append(d, ignore_index=True)
    df.to_csv('./Results/1DO-1DS.csv', index=False)